# Pieces of code(just to understand the process)

In [ ]:
!pip install scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=fd0d879cd6cd0bbf7eb36d7372509c2c29d003fd2de776452ea29dab98bf506d
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


Input/Outputs/Membership function/Fuzzy sets

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

Fever_universe = np.arange(36, 41.1, 0.1)
Fever_points = {'Low': [36, 36.5, 37.5], 'Medium': [37, 37.5, 38.5], 'High': [38, 38.5, 41]}
Fever = create_antecedent('Fever', Fever_universe, Fever_points)

Headache_universe = np.arange(0, 10.1, 1)
Headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Intense': [6, 8, 10]}
Headache = create_antecedent('Headache', Headache_universe, Headache_points)

Cough_universe = np.arange(0, 10.1, 1)
Cough_points = {'Occasional': [0, 2, 4], 'Frequent': [3, 5, 7], 'Persistent': [6, 8, 10]}
Cough = create_antecedent('Cough', Cough_universe, Cough_points)

Diagnosis_universe = np.arange(0, 8.1, 1)
Diagnosis_points = {'Unknown': [0, 2, 4], 'Mild_viral_infection': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Flu': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Bronchitis': [5, 6, 7],
                    'Pneumonia': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Respiratory_allergy': [8, 9, 10]}
Diagnosis = create_consequent('Diagnosis', Diagnosis_universe, Diagnosis_points)


Rules for each diagnostic

In [ ]:
# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Frequent']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Occasional']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Frequent']),
              consequent=Diagnosis['Bacterial_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Respiratory_allergy'])
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)

# Create the simulation for the control system
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)


Simulation setup

In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00


In [ ]:

input_fever = widgets.FloatText(value=37.0, description='Fever (°C):', step=0.1)
input_headache = widgets.IntSlider(value=5, min=0, max=10, description='Headache:')
input_cough = widgets.IntSlider(value=5, min=0, max=10, description='Cough:')

btn_simulate = widgets.Button(description="Diagnostic Simulation")

def interpret_diagnosis(value):
    if np.isnan(value):
        return "Unknown Diagnostic"
    elif value <= 2.5:
        return "Mild viral infection"
    elif 2.5 < value <= 4.5:
        return "Common cold"
    elif 4.5 < value <= 6.5:
        return "Flu"
    elif 6.5 < value <= 8.5:
        return "Sinusitis"
    elif 8.5 < value <= 10.5:
        return "Bronchitis"
    elif 10.5 < value <= 12.5:
        return "Pneumonia"
    elif 12.5 < value <= 14.5:
        return "Bacterian infection "
    elif 14.5 < value <= 16.5:
        return "Respiratory allergy"
    else:
        return "Inconclusive diagnosis"



def on_simulate_clicked(b):
    simulation_diagnostic.input['Fever'] = input_fever.value
    simulation_diagnostic.input['Headache'] = input_headache.value
    simulation_diagnostic.input['Cough'] = input_cough.value

    try:
        simulation_diagnostic.compute()
        diagnosis_text = interpret_diagnosis(simulation_diagnostic.output['Diagnosis'])
    except ValueError:
        diagnosis_text = "Inconclusive diagnosis"

    clear_output(wait=True)
    display(input_fever, input_headache, input_cough, btn_simulate)
    print(f"Suggested diagnosis: {diagnosis_text}")

btn_simulate.on_click(on_simulate_clicked)


display(input_fever, input_headache, input_cough, btn_simulate)


FloatText(value=37.0, description='Fever (°C):', step=0.1)

IntSlider(value=5, description='Headache:', max=10)

IntSlider(value=5, description='Cough:', max=10)

Button(description='Diagnostic Simulation', style=ButtonStyle())

In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Define symptom and diagnosis options
symptom_options = ['Fever', 'Headache', 'Cough']
diagnosis_options = ['Unknown', 'Mild viral infection', 'Common cold', 'Flu',
                     'Sinusitis', 'Bronchitis', 'Pneumonia', 'Bacterial infection',
                     'Respiratory allergy']

# Define interactive widgets with styling
symptom_dropdown = widgets.Dropdown(options=symptom_options, description='Symptom:',
                                     style={'description_width': 'initial', 'width': '300px'})
symptom_value = widgets.FloatSlider(value=37.0, min=36.0, max=41.0, step=0.1, description='Value:',
                                    style={'description_width': 'initial', 'width': '300px'})
diagnosis_dropdown = widgets.Dropdown(options=diagnosis_options, description='Diagnosis:',
                                      style={'description_width': 'initial', 'width': '300px'})
btn_add_symptom = widgets.Button(description="Add Symptom", button_style='success',
                                  layout={'width': '150px'})
btn_remove_symptom = widgets.Button(description="Remove Symptom", button_style='danger',
                                     layout={'width': '150px'})
btn_diagnose = widgets.Button(description="Diagnose", button_style='info', layout={'width': '150px'})
output_diagnosis = widgets.Output()

# Define list to store selected symptoms
selected_symptoms = []

# Define function to handle adding symptom
def add_symptom(b):
    selected_symptoms.append((symptom_dropdown.value, symptom_value.value))
    with output_diagnosis:
        clear_output(wait=True)
        print("Symptom added:", selected_symptoms)

# Define function to handle removing symptom
def remove_symptom(b):
    if selected_symptoms:
        selected_symptoms.pop()
        with output_diagnosis:
            clear_output(wait=True)
            print("Symptom removed:", selected_symptoms)

# Define function to handle diagnosis
def diagnose(b):
    if selected_symptoms:
        clear_output(wait=True)
        display(symptom_dropdown, symptom_value, btn_add_symptom, btn_remove_symptom, btn_diagnose)
        print("Selected Symptoms:")
        for symptom, value in selected_symptoms:
            print(f"- {symptom}: {value}")
        print("\nDiagnosis:")
        # Perform diagnostic inference here based on selected symptoms
        # Display diagnostic results using output_diagnosis widget

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)

# Display initial widgets with colorful styling
display(widgets.VBox([
    widgets.HBox([symptom_dropdown, symptom_value]),
    widgets.HBox([btn_add_symptom, btn_remove_symptom, btn_diagnose]),
    output_diagnosis
]), layout={'align_items': 'center', 'justify_content': 'center', 'width': '80%', 'margin': '0 auto'})


# First iteration


In [ ]:
def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

Fever_universe = np.arange(36, 41.1, 0.1)
Fever_points = {'Low': [36, 36.5, 37.5], 'Medium': [37, 37.5, 38.5], 'High': [38, 38.5, 41]}
Fever = create_antecedent('Fever', Fever_universe, Fever_points)

Headache_universe = np.arange(0, 10.1, 1)
Headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Intense': [6, 8, 10]}
Headache = create_antecedent('Headache', Headache_universe, Headache_points)

Cough_universe = np.arange(0, 10.1, 1)
Cough_points = {'Occasional': [0, 2, 4], 'Frequent': [3, 5, 7], 'Persistent': [6, 8, 10]}
Cough = create_antecedent('Cough', Cough_universe, Cough_points)

Diagnosis_universe = np.arange(0, 8.1, 1)
Diagnosis_points = {'Unknown': [0, 2, 4], 'Mild_viral_infection': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Flu': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Bronchitis': [5, 6, 7],
                    'Pneumonia': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Respiratory_allergy': [8, 9, 10]}
Diagnosis = create_consequent('Diagnosis', Diagnosis_universe, Diagnosis_points)

# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Frequent']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Occasional']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Frequent']),
              consequent=Diagnosis['Bacterial_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Respiratory_allergy'])
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)

# Create the simulation for the control system
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)


# Define symptom and diagnosis options
symptom_options = ['Fever', 'Headache', 'Cough']
diagnosis_options = ['Unknown', 'Mild viral infection', 'Common cold', 'Flu',
                     'Sinusitis', 'Bronchitis', 'Pneumonia', 'Bacterial infection',
                     'Respiratory allergy']

# Define interactive widgets with styling
symptom_dropdown = widgets.Dropdown(options=symptom_options, description='Symptom:',
                                     style={'description_width': 'initial', 'width': '300px'})
symptom_value = widgets.FloatSlider(value=37.0, min=36.0, max=41.0, step=0.1, description='Value:',
                                    style={'description_width': 'initial', 'width': '300px'})
diagnosis_dropdown = widgets.Dropdown(options=diagnosis_options, description='Diagnosis:',
                                      style={'description_width': 'initial', 'width': '300px'})
btn_add_symptom = widgets.Button(description="Add Symptom", button_style='success',
                                  layout={'width': '150px'})
btn_remove_symptom = widgets.Button(description="Remove Symptom", button_style='danger',
                                     layout={'width': '150px'})
btn_diagnose = widgets.Button(description="Diagnose", button_style='info', layout={'width': '150px'})
output_diagnosis = widgets.Output()

# Define list to store selected symptoms
selected_symptoms = []

# Define function to handle adding symptom
def add_symptom(b):
    selected_symptoms.append((symptom_dropdown.value, symptom_value.value))
    with output_diagnosis:
        clear_output(wait=True)
        print("Symptom added:", selected_symptoms)

# Define function to handle removing symptom
def remove_symptom(b):
    if selected_symptoms:
        selected_symptoms.pop()
        with output_diagnosis:
            clear_output(wait=True)
            print("Symptom removed:", selected_symptoms)

# Define function to handle diagnosis
def diagnose(b):
    if selected_symptoms:
        clear_output(wait=True)
        display(symptom_dropdown, symptom_value, btn_add_symptom, btn_remove_symptom, btn_diagnose)
        print("Selected Symptoms:")
        for symptom, value in selected_symptoms:
            print(f"- {symptom}: {value}")
        print("\nDiagnosis:")
        # Perform diagnostic inference here based on selected symptoms
        # Display diagnostic results using output_diagnosis widget

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)

# Display initial widgets with colorful styling
display(widgets.VBox([
    widgets.HBox([symptom_dropdown, symptom_value]),
    widgets.HBox([btn_add_symptom, btn_remove_symptom, btn_diagnose]),
    output_diagnosis
]), layout={'align_items': 'center', 'justify_content': 'center', 'width': '80%', 'margin': '0 auto'})



///////////////////////////////////////////////////////////////////////////


New improved version( still needs commentaries and check for improvements )

# Simple Implementation

In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from ipywidgets import widgets, HBox, VBox, Dropdown, FloatSlider, Button, Output, Text
from IPython.display import display, clear_output

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=31cfe993d548962a2e4899d5f1c6ad90e1f6ec6910e09acdabe8219710f26a7d
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


Updated code ( cant add the same symptom more than one time )

In [8]:


# Define functions to create membership functions, antecedents, and consequents
def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

# Define universes and membership points for each symptom and diagnosis
fever_universe = np.arange(36, 41.1, 0.1)
fever_points = {'Low': [36, 37, 38], 'Medium': [37.5, 38.5, 39.5], 'High': [39, 40, 41]}
fever = create_antecedent('fever', fever_universe, fever_points)

headache_universe = np.arange(0, 10.1, 1)
headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
headache = create_antecedent('headache', headache_universe, headache_points)

cough_universe = np.arange(0, 10.1, 1)
cough_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
cough = create_antecedent('cough', cough_universe, cough_points)

fatigue_universe = np.arange(0, 10.1, 1)
fatigue_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
fatigue = create_antecedent('fatigue', fatigue_universe, fatigue_points)

sore_throat_universe = np.arange(0, 10.1, 1)
sore_throat_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
sore_throat = create_antecedent('soreThroat', sore_throat_universe, sore_throat_points)

shortness_of_breath_universe = np.arange(0, 10.1, 1)
shortness_of_breath_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
shortness_of_breath = create_antecedent('shortnessOfBreath', shortness_of_breath_universe, shortness_of_breath_points)

nausea_universe = np.arange(0, 10.1, 1)
nausea_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
nausea = create_antecedent('nausea', nausea_universe, nausea_points)

diagnosis_universe = np.arange(0, 10.1, 1)
diagnosis_points = {'Unknown': [0, 1, 2], 'Respiratory_allergy': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Mild_viral_infection': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Flu': [5, 6, 7],
                    'Bronchitis': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Pneumonia': [8, 9, 10]}
diagnosis = create_consequent('diagnosis', diagnosis_universe, diagnosis_points)

# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(fever['Low'] & headache['Low'] & cough['Mild']),
              consequent=diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Low'] & cough['Moderate']),
              consequent=diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Moderate'] & cough['Mild']),
              consequent=diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Moderate'] & cough['Moderate']),
              consequent=diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(fever['Medium'] & headache['Moderate'] & cough['Moderate']),
              consequent=diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(fever['Medium'] & headache['Severe'] & cough['Mild']),
              consequent=diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(fever['Low'] & headache['Severe'] & cough['Mild']),
              consequent=diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(fever['Medium'] & headache['Moderate'] & cough['Severe']),
              consequent=diagnosis['Flu']),
    ctrl.Rule(antecedent=(fever['Medium'] & fatigue['Severe'] & shortness_of_breath['Moderate']),
              consequent=diagnosis['Flu']),
    ctrl.Rule(antecedent=(fever['Medium'] & nausea['Moderate'] & headache['Moderate']),
              consequent=diagnosis['Flu']),
    ctrl.Rule(antecedent=(fever['High'] & headache['Moderate'] & cough['Severe']),
              consequent=diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(fever['High'] & headache['Severe'] & cough['Severe']),
              consequent=diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(fever['High'] & shortness_of_breath['Severe'] & fatigue['Severe']),
              consequent=diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(fever['High'] & sore_throat['Severe'] & headache['Severe']),
              consequent=diagnosis['Bacterial_infection']),
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)

# Create the simulation for the control system
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)

# Define symptom and diagnosis options
symptom_options = ['Fever', 'Headache', 'Cough', 'Fatigue', 'Sore Throat', 'Shortness of Breath', 'Nausea']
diagnosis_options = ['Unknown', 'Respiratory allergy', 'Common cold', 'Mild viral infection',
                     'Sinusitis', 'Flu', 'Bronchitis', 'Bacterial infection', 'Pneumonia']

# Define interactive widgets with styling
symptom_dropdown = widgets.Dropdown(options=symptom_options, description='Symptom:',
                                     style={'description_width': 'initial', 'width': '300px'})
symptom_value = widgets.FloatSlider(value=37.0, min=36.0, max=41.0, step=0.1, description='Value:',
                                    style={'description_width': 'initial', 'width': '300px'})
btn_add_symptom = widgets.Button(description="Add Symptom", button_style='success',
                                  layout={'width': '150px'})
btn_remove_symptom = widgets.Button(description="Remove Symptom", button_style='danger',
                                     layout={'width': '150px'})
btn_diagnose = widgets.Button(description="Diagnose", button_style='info', layout={'width': '150px'})
output_diagnosis = widgets.Output()

# Define login widgets
login_text = widgets.Text(description="Username:", style={'description_width': 'initial', 'width': '300px'})
login_button = widgets.Button(description="Login", button_style='primary', layout={'width': '150px'})
logout_button = widgets.Button(description="Logout", button_style='warning', layout={'width': '150px'})

# Define list to store selected symptoms and user information
selected_symptoms = []
user = None
diagnostic_history = {}

# Function to update the value slider range based on selected symptom
def update_symptom_value_range(change):
    symptom = change['new']
    if symptom == 'Fever':
        symptom_value.min = 36.0
        symptom_value.max = 41.0
        symptom_value.step = 0.1
        symptom_value.value = 37.0
    else:
        symptom_value.min = 0.0
        symptom_value.max = 10.0
        symptom_value.step = 1.0
        symptom_value.value = 5.0

symptom_dropdown.observe(update_symptom_value_range, names='value')

# Define function to handle adding symptom
def add_symptom(b):
    if not user:
        with output_diagnosis:
            clear_output(wait=True)
            print("Please log in first.")
        return
    symptom = symptom_dropdown.value
    value = symptom_value.value
    if symptom not in [s[0] for s in selected_symptoms]:
        selected_symptoms.append((symptom, value))
        with output_diagnosis:
            clear_output(wait=True)
            print("Selected Symptoms:")
            for s, v in selected_symptoms:
                print(f"{s}: {v}°C" if s == "Fever" else f"{s}: {v}")
    else:
        with output_diagnosis:
            clear_output(wait=True)
            print(f"Symptom '{symptom}' already added.")

# Define function to handle removing symptom
def remove_symptom(b):
    if not user:
        with output_diagnosis:
            clear_output(wait=True)
            print("Please log in first.")
        return
    if selected_symptoms:
        removed_symptom = selected_symptoms.pop()
        with output_diagnosis:
            clear_output(wait=True)
            print(f"Symptom removed: {removed_symptom}")
            print("Remaining Symptoms:")
            for s, v in selected_symptoms:
                print(f"{s}: {v}°C" if s == "Fever" else f"{s}: {v}")

# Define function to handle diagnosis
def diagnose(b):
    if not user:
        with output_diagnosis:
            clear_output(wait=True)
            print("Please log in first.")
        return
    if selected_symptoms:
        with output_diagnosis:
            clear_output(wait=True)
            print("Selected Symptoms:")
            for symptom, value in selected_symptoms:
                print(f"- {symptom}: {value}")
                key = symptom.lower().replace(" ", "")
                simulation_diagnostic.input[key] = value
            simulation_diagnostic.compute()
            diagnosis_result = simulation_diagnostic.output['diagnosis']
            print("Diagnosis result:", diagnosis_result)
            # Save the diagnostic result
            if user not in diagnostic_history:
                diagnostic_history[user] = []
            diagnostic_history[user].append({'symptoms': selected_symptoms.copy(), 'diagnosis': diagnosis_result})
            print("Diagnostic result saved.")

# Define function for login
def login(b):
    global user
    username = login_text.value
    if username:
        user = username
        with output_diagnosis:
            clear_output(wait=True)
            print(f"Logged in as: {user}")
        display(logout_button, HBox([symptom_dropdown, symptom_value]), HBox([btn_add_symptom, btn_remove_symptom]), btn_diagnose)
    else:
        with output_diagnosis:
            clear_output(wait=True)
            print("Please enter a username.")

# Define function for logout
def logout(b):
    global user
    user = None
    selected_symptoms.clear()
    with output_diagnosis:
        clear_output(wait=True)
        print("Logged out.")
    display(login_text, login_button)

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)
login_button.on_click(login)
logout_button.on_click(logout)

# Display login widgets initially
display(VBox([
    login_text,
    login_button,
    output_diagnosis
]))


Button(button_style='warning', description='Logout', layout=Layout(width='150px'), style=ButtonStyle())

Button(button_style='info', description='Diagnose', layout=Layout(width='150px'), style=ButtonStyle())

Intuitionistic approach

# Intuitionistic approach

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from ipywidgets import widgets, HBox, VBox, Dropdown, FloatSlider, Button, Output, interactive
from IPython.display import display, clear_output

In [ ]:


# Define Intuitionistic Fuzzy Set
class IntuitionisticFuzzySet:
    def __init__(self, universe, membership, non_membership):
        self.universe = universe
        self.membership = membership
        self.non_membership = non_membership

    def plot(self):
        plt.figure()
        plt.plot(self.universe, self.membership, 'b', linewidth=1.5, label='Membership')
        plt.plot(self.universe, self.non_membership, 'r', linewidth=1.5, label='Non-membership')
        plt.fill_between(self.universe, self.membership, self.non_membership, color='gray', alpha=0.2)
        plt.title('Intuitionistic Fuzzy Set')
        plt.legend()
        plt.show()

# Define membership function
def define_membership_function(universe, points):
    membership = fuzz.trimf(universe, points)
    non_membership = 1 - membership
    return IntuitionisticFuzzySet(universe, membership, non_membership)

# Create antecedents and consequents
def create_antecedent(name, universe, points_dict):
    antecedent = {}
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = {}
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

# Define universes and membership points
Fever_universe = np.arange(36, 41.1, 0.1)
Fever_points = {'Low': [36, 37, 38], 'Medium': [37.5, 38.5, 39.5], 'High': [39, 40, 41]}
Fever = create_antecedent('Fever', Fever_universe, Fever_points)

Headache_universe = np.arange(0, 10.1, 1)
Headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
Headache = create_antecedent('Headache', Headache_universe, Headache_points)

Cough_universe = np.arange(0, 10.1, 1)
Cough_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
Cough = create_antecedent('Cough', Cough_universe, Cough_points)

Fatigue_universe = np.arange(0, 10.1, 1)
Fatigue_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
Fatigue = create_antecedent('Fatigue', Fatigue_universe, Fatigue_points)

SoreThroat_universe = np.arange(0, 10.1, 1)
SoreThroat_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
SoreThroat = create_antecedent('SoreThroat', SoreThroat_universe, SoreThroat_points)

ShortnessOfBreath_universe = np.arange(0, 10.1, 1)
ShortnessOfBreath_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
ShortnessOfBreath = create_antecedent('ShortnessOfBreath', ShortnessOfBreath_universe, ShortnessOfBreath_points)

Nausea_universe = np.arange(0, 10.1, 1)
Nausea_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
Nausea = create_antecedent('Nausea', Nausea_universe, Nausea_points)

Diagnosis_universe = np.arange(0, 10.1, 1)
Diagnosis_points = {'Unknown': [0, 1, 2], 'Respiratory_allergy': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Mild_viral_infection': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Flu': [5, 6, 7],
                    'Bronchitis': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Pneumonia': [8, 9, 10]}
Diagnosis = create_consequent('Diagnosis', Diagnosis_universe, Diagnosis_points)

# Define rules
rules = [
    {'antecedents': {'Fever': 'Low', 'Headache': 'Low', 'Cough': 'Mild'}, 'consequent': 'Respiratory_allergy'},
    {'antecedents': {'Fever': 'Low', 'Headache': 'Low', 'Cough': 'Moderate'}, 'consequent': 'Common_cold'},
    {'antecedents': {'Fever': 'Low', 'Headache': 'Moderate', 'Cough': 'Mild'}, 'consequent': 'Common_cold'},
    {'antecedents': {'Fever': 'Low', 'Headache': 'Moderate', 'Cough': 'Moderate'}, 'consequent': 'Common_cold'},
    {'antecedents': {'Fever': 'Medium', 'Headache': 'Moderate', 'Cough': 'Moderate'}, 'consequent': 'Mild_viral_infection'},
    {'antecedents': {'Fever': 'Medium', 'Headache': 'Severe', 'Cough': 'Mild'}, 'consequent': 'Mild_viral_infection'},
    {'antecedents': {'Fever': 'Low', 'Headache': 'Severe', 'Cough': 'Mild'}, 'consequent': 'Sinusitis'},
    {'antecedents': {'Fever': 'Medium', 'Headache': 'Moderate', 'Cough': 'Severe'}, 'consequent': 'Flu'},
    {'antecedents': {'Fever': 'Medium', 'Headache': 'Severe', 'Cough': 'Moderate'}, 'consequent': 'Flu'},
    {'antecedents': {'Fever': 'Medium', 'Headache': 'Moderate', 'Cough': 'Moderate'}, 'consequent': 'Flu'},
    {'antecedents': {'Fever': 'High', 'Headache': 'Moderate', 'Cough': 'Severe'}, 'consequent': 'Bronchitis'},
    {'antecedents': {'Fever': 'High', 'Headache': 'Severe', 'Cough': 'Severe'}, 'consequent': 'Pneumonia'},
    {'antecedents': {'Fever': 'High', 'Headache': 'Severe', 'Cough': 'Severe'}, 'consequent': 'Pneumonia'},
    {'antecedents': {'Fever': 'High', 'Headache': 'Severe', 'Cough': 'Severe'}, 'consequent': 'Bacterial_infection'},
]

# Store selected symptoms
selected_symptoms = []

# Diagnosis function
def diagnose(selected_symptoms):
    diagnosis_results = {}
    for symptom, value in selected_symptoms:
        antecedent_sets = eval(symptom)
        diagnosis_results[symptom] = {}
        for label, if_set in antecedent_sets.items():
            membership_value = fuzz.interp_membership(if_set.universe, if_set.membership, value)
            non_membership_value = fuzz.interp_membership(if_set.universe, if_set.non_membership, value)
            diagnosis_results[symptom][label] = (membership_value, non_membership_value)
            print(f"Symptom: {symptom}, Value: {value}, Label: {label}, Membership: {membership_value}, Non-membership: {non_membership_value}")
    return diagnosis_results

# Aggregate results to get final diagnosis
def aggregate_diagnosis(diagnosis_results):
    diagnosis_aggregates = {diag: [] for diag in Diagnosis.keys()}
    for rule in rules:
        membership_values = []
        non_membership_values = []
        for symptom, label in rule['antecedents'].items():
            if symptom in diagnosis_results and label in diagnosis_results[symptom]:
                membership, non_membership = diagnosis_results[symptom][label]
                membership_values.append(membership)
                non_membership_values.append(non_membership)
        if membership_values and non_membership_values:
            min_membership = min(membership_values)
            min_non_membership = min(non_membership_values)
            diagnosis_aggregates[rule['consequent']].append((min_membership, min_non_membership))

    final_diagnosis = {}
    for diagnosis, values in diagnosis_aggregates.items():
        if values:
            membership_values, non_membership_values = zip(*values)
            final_diagnosis[diagnosis] = (max(membership_values), min(non_membership_values))
    return final_diagnosis

# Widgets
symptom_dropdown = Dropdown(options=['Fever', 'Headache', 'Cough', 'Fatigue', 'SoreThroat', 'ShortnessOfBreath', 'Nausea'], description='Symptom:')
symptom_value_slider = FloatSlider(description='Value:', min=0, max=10, step=1, value=5, layout={'width': '400px'})
add_symptom_button = Button(description='Add Symptom', button_style='success', layout={'width': '150px'})
output = Output()

# Function to update value slider range
def update_symptom_value_range(change):
    symptom = change['new']
    if symptom == 'Fever':
        symptom_value_slider.min = 36.0
        symptom_value_slider.max = 41.0
        symptom_value_slider.step = 0.1
        symptom_value_slider.value = 37.0
    else:
        symptom_value_slider.min = 0.0
        symptom_value_slider.max = 10.0
        symptom_value_slider.step = 1.0
        symptom_value_slider.value = 5.0

symptom_dropdown.observe(update_symptom_value_range, names='value')

# Add symptom function
def add_symptom(b):
    symptom = symptom_dropdown.value
    value = symptom_value_slider.value
    if symptom not in [s for s, v in selected_symptoms]:
        selected_symptoms.append((symptom, value))
        with output:
            clear_output(wait=True)
            print("Selected Symptoms:")
            for s, v in selected_symptoms:
                print(f"{s}: {v}°C" if s == "Fever" else f"{s}: {v}")
    else:
        with output:
            clear_output(wait=True)
            print(f"{symptom} has already been added.")

# Diagnose function
def diagnose_and_display(b):
    diagnosis_results = diagnose(selected_symptoms)
    final_diagnosis = aggregate_diagnosis(diagnosis_results)
    with output:
        clear_output(wait=True)
        print("Diagnosis Results:")
        for diag, (mem, non_mem) in final_diagnosis.items():
            print(f"{diag} - Membership: {mem:.2f}, Non-membership: {non_mem:.2f}")

# Bind buttons
add_symptom_button.on_click(add_symptom)
diagnose_button = Button(description='Diagnose', button_style='info')
diagnose_button.on_click(diagnose_and_display)

# Display widgets
display(VBox([symptom_dropdown, symptom_value_slider, add_symptom_button, diagnose_button, output]))


Symptom: Fever, Value: 5.0, Label: Low, Membership: 0.0, Non-membership: 0.0
Symptom: Fever, Value: 5.0, Label: Medium, Membership: 0.0, Non-membership: 0.0
Symptom: Fever, Value: 5.0, Label: High, Membership: 0.0, Non-membership: 0.0
Symptom: Fatigue, Value: 5.0, Label: Low, Membership: 0.0, Non-membership: 1.0
Symptom: Fatigue, Value: 5.0, Label: Moderate, Membership: 1.0, Non-membership: 0.0
Symptom: Fatigue, Value: 5.0, Label: Severe, Membership: 0.0, Non-membership: 1.0
Symptom: Nausea, Value: 5.0, Label: Mild, Membership: 0.0, Non-membership: 1.0
Symptom: Nausea, Value: 5.0, Label: Moderate, Membership: 1.0, Non-membership: 0.0
Symptom: Nausea, Value: 5.0, Label: Severe, Membership: 0.0, Non-membership: 1.0
